# Import libraries

In [68]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, accuracy_score
from joblib import dump, load

from tensorflow import keras
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, Flatten,\
    concatenate,SimpleRNN,LSTM,Embedding,GRU,Bidirectional,Masking

# Data marshalling

In [2]:
ptbdb_normal = pd.read_csv("ptbdb_normal.csv", header=None)
ptbdb_abnormal = pd.read_csv("ptbdb_abnormal.csv", header=None)
ptbdb = pd.concat([ptbdb_normal, ptbdb_abnormal])

mitbih_train = pd.read_csv("mitbih_train.csv", header=None)
mitbih_train = mitbih_train.sample(frac=1)
mitbih_test = pd.read_csv("mitbih_test.csv", header=None)


ptbdb_categories=2
ptbdb_train, ptbdb_test = train_test_split(ptbdb, test_size=0.2, random_state=42, stratify=ptbdb[187])
ptbdb_train_Y = np.array(ptbdb_train[187].values).astype(np.int8)
ptbdb_train_X = np.array(ptbdb_train[list(range(187))].values)[..., np.newaxis]
ptbdb_validation_x, ptbdb_train_x, ptbdb_validation_y, ptbdb_train_y = train_test_split(ptbdb_train_X, ptbdb_train_Y, test_size=0.33, random_state=42)
ptbdb_test_Y = np.array(ptbdb_test[187].values).astype(np.int8)
ptbdb_test_X = np.array(ptbdb_test[list(range(187))].values)[..., np.newaxis]

mitbih_categories=5
mitbih_train_Y = np.array(mitbih_train[187].values).astype(np.int8)
mitbih_train_X = np.array(mitbih_train[list(range(187))].values)[..., np.newaxis]
mitbih_validation_x, mitbih_train_x, mitbih_validation_y, mitbih_train_y = train_test_split(mitbih_train_X, mitbih_train_Y, test_size=0.33, random_state=42)


Hyperparameters:
|-dropout: 0.30000000000000004
|-gate_type: GRU
|-learning_rate: 0.01
|-num_layers: 1
|-optimizer: ADAM
|-recurrent_dropout: 0.2
|-tuner/bracket: 3
|-tuner/epochs: 5
|-tuner/initial_epoch: 2
|-tuner/round: 1
|-tuner/trial_id: 1be35566db3f7b602560eef664260300
|-units: 137

# Train base

In [3]:
print('hi')
units=137
num_layers=1
gate_type='GRU'
learning_rate=0.01
dropout_rate=0.3
optimizer='ADAM'
recurrent_dropout_rate=0.2
model = keras.Sequential()
model.add(keras.layers.Masking(mask_value=0.,input_shape=(187, 1)))
for i in range(num_layers):
    if gate_type=='SimpleRNN':
        model.add(SimpleRNN(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    elif gate_type=='GRU':
        model.add(GRU(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    elif gate_type=='LSTM':
        model.add(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    elif gate_type=='Bidirectional':
        model.add(Bidirectional(LSTM(units,return_sequences=True,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
if gate_type=='SimpleRNN':
    model.add(SimpleRNN(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
elif gate_type=='GRU':
    model.add(GRU(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
elif gate_type=='LSTM':
    model.add(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
elif gate_type=='Bidirectional':
    model.add(Bidirectional(LSTM(units,dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate)))
model.add(Dense(64, activation=activations.relu, name="dense_1"))
model.add(Dense(64, activation=activations.relu, name="dense_2"))
model.add(Dense(4, activation=activations.softmax, name="dense_3_mitbih"))
optimizer_type=optimizer
learning_rate=learning_rate
if optimizer_type=='SGD':
    optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
elif optimizer_type=='ADAM':
    optimizer=keras.optimizers.Adam(
        learning_rate=learning_rate)
elif optimizer_type=='RMSprop':
    optimizer=keras.optimizers.RMSprop(
        learning_rate=learning_rate)
loss=losses.sparse_categorical_crossentropy

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['acc'])
file_path = "baseline_rnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
model.build()
model.summary()
model.fit(mitbih_train_X, mitbih_train_Y, epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1,validation_steps=100)

hi
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 187, 1)            0         
_________________________________________________________________
gru (GRU)                    (None, 187, 137)          57540     
_________________________________________________________________
gru_1 (GRU)                  (None, 137)               113436    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8832      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3_mitbih (Dense)       (None, 4)                 260       
Total params: 184,228
Trainable params: 184,228
Non-trainable params: 0
_______________________________________________

In [22]:
model1 = keras.models.Sequential(model.layers)#since I already popped three times, otherwise use model.layers[:-3]
model1.build(input_shape=(187, 1))
model1.summary()
model1.save('transferral_base.h5')

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 187, 1)            0         
_________________________________________________________________
gru (GRU)                    (None, 187, 137)          57540     
_________________________________________________________________
gru_1 (GRU)                  (None, 137)               113436    
Total params: 170,976
Trainable params: 170,976
Non-trainable params: 0
_________________________________________________________________


# First part of the first transferral model

In [35]:
predictions_train_X=model1.predict(ptbdb_train_X)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [47]:
predictions_train_X=np.expand_dims(predictions_train_X, axis=2)


# Second part of first model

In [73]:
model2=keras.Sequential()
model2.add(Input(shape=(137,1)))
model2.add(Dense(64, activation=activations.relu, name="dense_1"))
model2.add(Dense(64, activation=activations.relu, name="dense_2"))
model2.add(Flatten())
model2.add(Dense(1, activation=activations.sigmoid, name="dense_3_ptbdb"))
if optimizer_type=='SGD':
    optimizer=keras.optimizers.SGD(learning_rate=learning_rate)
elif optimizer_type=='ADAM':
    optimizer=keras.optimizers.Adam(
        learning_rate=learning_rate)
elif optimizer_type=='RMSprop':
    optimizer=keras.optimizers.RMSprop(
        learning_rate=learning_rate)
loss=losses.binary_crossentropy

model2.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['acc'])
file_path = "transferral_model2.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early, redonplat]  # early
model2.build()
model2.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 137, 64)           128       
_________________________________________________________________
dense_2 (Dense)              (None, 137, 64)           4160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 8768)              0         
_________________________________________________________________
dense_3_ptbdb (Dense)        (None, 1)                 8769      
Total params: 13,057
Trainable params: 13,057
Non-trainable params: 0
_________________________________________________________________


In [74]:
model2.fit(predictions_train_X,ptbdb_train_Y)
model2.save('transfer2.h5')

Train on 11641 samples
11641/11641 [==============================] - 2s 157us/sample - loss: 0.6028 - acc: 0.7200


# Second model

In [79]:
model1.add(Dense(64, activation=activations.relu,name='dense_10'))
model1.add(Dense(64, activation=activations.relu,name='dense_11'))
model1.add(Flatten())
model1.add(Dense(1, activation=activations.sigmoid,name='dense_12'))

model1.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['acc'])
file_path = "model3.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early, redonplat]  # early
model1.build()
model1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 187, 1)            0         
_________________________________________________________________
gru (GRU)                    (None, 187, 137)          57540     
_________________________________________________________________
gru_1 (GRU)                  (None, 137)               113436    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8832      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                

In [81]:
model1.fit(ptbdb_train_X,ptbdb_train_Y)
model1.save('transfer3.h5')

Train on 11641 samples
11641/11641 [==============================] - 241s 21ms/sample - loss: 0.5952 - acc: 0.7206


# Third model

In [95]:
model4= keras.models.load_model("baseline_rnn_mitbih.h5")
model4= keras.models.Sequential(model4.layers[:3])
model4.trainable=False
model4.add(Dense(64, activation=activations.relu,name='dense_10'))
model4.add(Dense(64, activation=activations.relu,name='dense_11'))
model4.add(Flatten())
model4.add(Dense(1, activation=activations.sigmoid,name='dense_12'))
model4.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['acc'])
file_path = "model4.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint, early, redonplat]  # early
model4.build()
model4.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 187, 1)            0         
_________________________________________________________________
gru (GRU)                    (None, 187, 137)          57540     
_________________________________________________________________
gru_1 (GRU)                  (None, 137)               113436    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8832      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
flatten_11 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)               

In [96]:
model4.fit(ptbdb_train_X,ptbdb_train_Y)
model4.save('transfer4.h5')

Train on 11641 samples
11641/11641 [==============================] - 242s 21ms/sample - loss: 0.5932 - acc: 0.7218
